In [47]:
!pip install datasets transformers==4.9.2 seqeval -q
!pip install sentencepiece -q

In [48]:
encoder = "airesearch/wangchanberta-base-att-spm-uncased"
batch_size = 16
num_epoch = 50
save_strategy = "no"

In [2]:
import pandas as pd
from training import train_classifier
from evaluation import evaluate, create_eval_dataset
from ast import literal_eval
from warnings import filterwarnings
filterwarnings("ignore")

In [52]:
train = pd.read_csv("our_model/train.tsv", sep='\t')
dev = pd.read_csv("our_model/dev.tsv", sep='\t')
test = pd.read_csv("our_model/test.tsv", sep='\t')

In [54]:
heads_10 = [str(i) for i in range(-10, 10+1)] + ["OUT_OF_RANGE"]
heads_5 = [str(i) for i in range(-5, 5+1)] + ["OUT_OF_RANGE"]
dep_types = [
    "case","obj","nsubj","compound","obl","advmod","root","acl","aux","xcomp",
    "conj","amod","cc","acl:relcl","mark","flat:name","nmod","nmod:poss","cop",
    "det","nummod","compound:prt","appos","advcl","fixed","ccomp","punct","obl:tmod",
    "clf","dep","obl:poss","aux:pass","nsubj:pass","csubj","discourse","cc:preconj",
    "parataxis","goeswith","iobj","dislocated","reparandum","vocative"
]

HEAD_CLASSIFIER_10 = train_classifier(
    train=train,
    dev=dev,
    test=test,
    label_name="rel_heads",
    label_list=heads_10,
    window_length=10,
    encoder=encoder,
    batch_size=batch_size,
    num_epoch=num_epoch,
    save_strategy=save_strategy
)

HEAD_CLASSIFIER_5 = train_classifier(
    train=train,
    dev=dev,
    test=test,
    label_name="rel_heads",
    label_list=heads_5,
    window_length=5,
    encoder=encoder,
    batch_size=batch_size,
    num_epoch=num_epoch,
    save_strategy=save_strategy
)

LABEL_CLASSIFIER = train_classifier(
    train=train,
    dev=dev,
    test=test,
    label_name="dep_types",
    label_list=dep_types,
    window_length=None,
    encoder=encoder,
    batch_size=batch_size,
    num_epoch=num_epoch,
    save_strategy=save_strategy
)

In [86]:
labelize = lambda x: [h if h == "OUT_OF_RANGE" or -10 <= int(h) <= 10 else "OUT_OF_RANGE" for h in literal_eval(x)]
head_dataset, label_dataset = create_eval_dataset(train, dev, test, labelize, heads_10, dep_types, encoder)
evaluate(HEAD_CLASSIFIER_10, heads_10, head_dataset, LABEL_CLASSIFIER, dep_types, label_dataset)

The following columns in the test set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: rel_heads, tokens, word_ids, tokens_bert.
***** Running Prediction *****
  Num examples = 158
  Batch size = 16


/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: -1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: -2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: -5 s

(78.69051227644741, 0)

In [ ]:
labelize = lambda x: [h if h == "OUT_OF_RANGE" or -5 <= int(h) <= 5 else "OUT_OF_RANGE" for h in literal_eval(x)]
head_dataset, label_dataset = create_eval_dataset(train, dev, test, labelize, heads_5, dep_types, encoder)
evaluate(HEAD_CLASSIFIER_5, heads_5, head_dataset, LABEL_CLASSIFIER, dep_types, label_dataset)